In [22]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torchmetrics

In [23]:
# mean and sd for images size 299,299
mean = [0.1854, 0.1854, 0.1855]
std = [0.2005, 0.2005, 0.2005]

batch_size = 32
model_path = '../models/inception_v3_5.pth'

# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
data_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=mean, std=std)
])
test_dataset = datasets.ImageFolder(root="../data/Testing", transform=data_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
# Load Inception V3 Model
model = models.inception_v3(weights=False)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, len(test_dataset.classes))
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

RuntimeError: Error(s) in loading state_dict for Inception3:
	Unexpected key(s) in state_dict: "AuxLogits.conv0.conv.weight", "AuxLogits.conv0.bn.weight", "AuxLogits.conv0.bn.bias", "AuxLogits.conv0.bn.running_mean", "AuxLogits.conv0.bn.running_var", "AuxLogits.conv0.bn.num_batches_tracked", "AuxLogits.conv1.conv.weight", "AuxLogits.conv1.bn.weight", "AuxLogits.conv1.bn.bias", "AuxLogits.conv1.bn.running_mean", "AuxLogits.conv1.bn.running_var", "AuxLogits.conv1.bn.num_batches_tracked", "AuxLogits.fc.weight", "AuxLogits.fc.bias". 

In [26]:
accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=len(test_dataset.classes)).to(device)
auc_metric = torchmetrics.AUROC(task="multiclass", num_classes=len(test_dataset.classes)).to(device)
confusion_matrix_metric = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=len(test_dataset.classes)).to(device)

In [27]:
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        
        _, preds = torch.max(outputs, 1)

        accuracy_metric.update(preds, labels)

        auc_metric.update(outputs, labels)

        confusion_matrix_metric.update(outputs, labels)
final_accuracy = accuracy_metric.compute()
final_auc = auc_metric.compute()
final_confusion_matrix = confusion_matrix_metric.compute()

print(f"Test Accuracy: {final_accuracy:.4f}")
print(f"Test AUC: {final_auc:.4f}")
print(f"Confusion Matrix:\n{final_confusion_matrix}")

Test Accuracy: 0.3089
Test AUC: 0.5712
Confusion Matrix:
tensor([[  0,   0, 300,   0],
        [  0,   0, 306,   0],
        [  0,   0, 405,   0],
        [  0,   0, 300,   0]], device='cuda:0')
